In [1]:
import ee
import geemap
import os
import numpy as np
from scipy import signal as sp
from matplotlib import pyplot as plt

In [ ]:
manaus_shp = '/home/6ru/tile_shapefiles/manaus/venus_fgmanaus_footprint.shp'
manaus = geemap.shp_to_ee(manaus_shp)
info = manaus.geometry()
manaus = ee.Geometry(info)

In [ ]:
atto_shp = '/Users/morgansteckler/Downloads/venus_atto_footprint/venus_atto_footprint.shp'
atto = geemap.shp_to_ee(atto_shp)
info = atto.geometry()
atto = ee.Geometry(info)

In [ ]:
corumba_shp = '/Users/morgansteckler/Downloads/venus_corumba_footprint/venus_corumba_footprint.shp'
corumba = geemap.shp_to_ee(corumba_shp)
info = corumba.geometry()
corumba = ee.Geometry(info)

In [ ]:
mato_shp = '/Users/morgansteckler/Downloads/venus_mato_footprint/venus_mato_footprint.shp'
mato = geemap.shp_to_ee(mato_shp)
info = mato.geometry()
mato = ee.Geometry(info)

In [ ]:
saop_shp = '/Users/morgansteckler/Downloads/venus_saop_footprint/venus_saop_footprint.shp'
saop = geemap.shp_to_ee(saop_shp)
info = saop.geometry()
saop = ee.Geometry(info)

In [ ]:
#IMPORTANT
#Select geom of interest
fc = manaus

In [ ]:
Map = geemap.Map()
Map.centerObject(fc, 10)
Map

In [ ]:
#Calculate NDRE and set palette
def addNDRE(image):
    ndre = image.normalizedDifference(['B8', 'B5']).rename('NDRE')
    return image.addBands(ndre)

#Set NDRE parameters for visualization
ndre_params = ({'min': 0.0,'max': 1.0, 'palette' : 
               ['FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718',
               '74A901', '66A000', '529400', '3E8601', '207401', '056201',
               '004C00', '023B01', '012E01', '011D01', '011301']})

#Create cloud mask
cloudBitMask = ee.Number(2).pow(10).int() #bit 10 is clouds
cirrusBitMask = ee.Number(2).pow(11).int() #bit 11 is cirrus clouds
def maskS2clouds(image):
    qa = image.select('QA60')
    # Both flags should be set to zero, indicating clear conditions.
    mask = (qa.bitwiseAnd(cloudBitMask).eq(0).And(
             qa.bitwiseAnd(cirrusBitMask).eq(0)))
    return image.updateMask(mask)

In [ ]:
#Load sentinel-2 data
s2 = (ee.ImageCollection('COPERNICUS/S2')#Select S2 1-C level data
      .select(['B5', 'B8', 'QA60']) #Select necessary bands to reduce memory usage
      .filterBounds(fc) #Narrow data spatially
      .filterDate(ee.Date('2016-01-01'), ee.Date('2019-12-31')) #Select years of interest
      .sort('CLOUDY_PIXEL_PERCENTAGE', False) #Put smallest %cloud image on top of stack
      .map(addNDRE)) #calculate NDRE and add as band

In [ ]:
#Create collection with cloud mask
col = s2.map(maskS2clouds)

In [ ]:
#15-day step
years = ee.List.sequence(2016, 2019)
step = ee.List.sequence(1, 365, 15)

def byYear(y):
    y = ee.Number(y)
    def byStep(d):
        d = ee.Number(d)
        return (col
                .filter(ee.Filter.calendarRange(y, y.add(1), 'year')) #yearly step
                .filter(ee.Filter.calendarRange(d, d.add(15), 'day_of_year')) #15-day step
                .qualityMosaic('NDRE') #Max greenness
                .set('step', [d, y])) #Add properties
    return step.map(byStep)

col = ee.ImageCollection.fromImages(years.map(byYear).flatten())
col = col.map(lambda image: image.clip(fc)).select('NDRE')

In [ ]:
#Makes sure there are 100 (25 15-day steps * 4 years = 100 composites)
count = col.size().getInfo()
print(count)

In [ ]:
#Check step property
properties = col.aggregate_array('step').getInfo()
print(properties)

In [ ]:
#Check that only one band is selected
bands = col.first().bandNames().getInfo()
print(bands)

In [ ]:
geom = fc

# Mean
def region_mean_Image(img):
    # mean over all pixels in the region for the image
    return img.set('mean', img.reduceRegion(ee.Reducer.mean(), geometry=geom, scale=20)) 

def region_mean_ImageCollection(ic):
    # mean over all pixels in the region, for each image in the ImageCollection
    stats = ic.map(region_mean_Image)
    stats_list = stats.aggregate_array('mean')
    return np.array(stats_list.getInfo())

# Median
def region_median_Image(img):
    # mean over all pixels in the region for the image
    return img.set('median', img.reduceRegion(ee.Reducer.median(), geometry=geom, scale=20)) 

def region_median_ImageCollection(ic):
    # mean over all pixels in the region, for each image in the ImageCollection
    stats = ic.map(region_median_Image)
    stats_list = stats.aggregate_array('median')
    return np.array(stats_list.getInfo())

# Stddev
def region_stddev_Image(img):
    # stddev over all pixels in the region for the image
    return img.set('stddev', img.reduceRegion(ee.Reducer.stdDev(), geometry=geom, scale=20)) 

def region_stddev_ImageCollection(ic):
    # stddev over all pixels in the region, for each image in the ImageCollection
    stats = ic.map(region_stddev_Image)
    stats_list = stats.aggregate_array('stddev')
    return np.array(stats_list.getInfo())

# Percentiles
def region_percentiles_Image(img):
    # 5,10,25,75,90,95 percentiles over all pixels in the region for the image
    return img.set('percentiles', img.reduceRegion(ee.Reducer.percentile([5,10,25,75,90,95]), geometry=geom, scale=20)) #) mean, median, stdDev, percentiles

def region_percentiles_ImageCollection(ic):
    # percentiles over all pixels in the region, for each image in the ImageCollection
    stats = ic.map(region_percentiles_Image)
    stats_list = stats.aggregate_array('percentiles')
    return np.array(stats_list.getInfo())

In [ ]:
#Create list of dictionaries as [{'NDRE': value} ...]
mean_list = region_mean_ImageCollection(col)
median_list = region_median_ImageCollection(col)
stddev_list = region_stddev_ImageCollection(col)
percentile_list = region_percentiles_ImageCollection(col)

In [ ]:
#Show if there are any blank {} dictionaries
print(mean_list)

In [ ]:
#Replace empty dicts with {'NDRE': None}
#Atto: 5, 24
#Saop: 9, 24, 99
dic = {'NDRE': None}
p_dic = {'NDRE_p10': None, 'NDRE_p25': None, 'NDRE_p5': None, 'NDRE_p75': None, 'NDRE_p90': None, 'NDRE_p95': None}
mean_list[5], mean_list[24] = dic, dic
median_list[5], median_list[24] = dic, dic
stddev_list[5], stddev_list[24] = dic, dic
percentile_list[5], percentile_list[24] = p_dic, p_dic

In [ ]:
#Take only the ndre mean, med, stddev dict values and append to new list
def newList(alist):
    list_name = []
    for value in alist:
        list_name.append(value['NDRE'])
    return list_name

#Append ndre percentile dict values to new lists
def p_list(alist):
    p10, p25, p5, p75, p90, p95 = [], [], [], [], [], []
    for value in alist:
        p5.append(value['NDRE_p5'])
        p10.append(value['NDRE_p10'])
        p25.append(value['NDRE_p25'])
        p75.append(value['NDRE_p75'])
        p90.append(value['NDRE_p90'])
        p95.append(value['NDRE_p95'])
    return p5, p10, p25, p75, p90, p95

In [ ]:
#Make lists for each set of values
mean_ndre = newList(mean_list)
median_ndre = newList(median_list)
stddev_ndre = newList(stddev_list)
percentile_ndre = p_list(percentile_list)

In [ ]:
#Change None values to np.nan so graphs will skip those Nulls
mean_ndre = [np.nan if v is None else v for v in mean_ndre]
median_ndre = [np.nan if v is None else v for v in median_ndre]
stddev_ndre = [np.nan if v is None else v for v in stddev_ndre]
percentile_ndre = [np.nan if v is None else v for v in percentile_ndre]

In [ ]:
#Make lists with +/- SD
plus_error = [x + y for x,y in zip(mean_ndre, stddev_ndre)]
minus_error = [x - y for x,y in zip(mean_ndre, stddev_ndre)]

In [ ]:
#Create 15-step list for four years
period = list(range(0,100))
period = np.array(period)

In [ ]:
#Manaus
#Mean and SD
x = period
y = mean_ndre
m = median_ndre
plt.axes([2, 0, 2, 1])
plt.ylim(bottom = 0, top = 1)
plt.grid(b = True, which = 'major', axis = 'x')
plt.title('2016-2019 (15-day step) mean, median, and SD of NDRE for Manaus Tile')
plt.fill_between(x, plus_error, minus_error, color = "grey", alpha = .5, label = "Standard Deviation")
mean = plt.plot(x,y, color = "black", label = "Mean")
median = plt.plot(x,m, color = "red", label = "Median")
plt.legend(loc = "lower right")
plt.xticks(ticks = [0, 25, 50, 75, 99], labels = ['Jan 2016', 'Jan 2017', 'Jan 2018', 'Jan 2019', '26 Dec 2019'])
plt.show()

In [ ]:
#Manaus
#Median and Percentiles
x = period
plt.axes([2, 0, 2, 1])
plt.ylim(bottom = 0, top = 1)
plt.grid(b = True, which = 'major', axis = 'x')
y5 = plt.plot(x,percentile_ndre[0], color = "red", label = "P5")
y10 = plt.plot(x,percentile_ndre[1], color = "orange", label = "P10")
y25 = plt.plot(x,percentile_ndre[2], color = "green", label = "P25")
y75 = plt.plot(x,percentile_ndre[3], color = "blue", label = "P75")
y90 = plt.plot(x,percentile_ndre[4], color = "purple", label = "P90")
y95 = plt.plot(x,percentile_ndre[5], color = "black", label = "P95")
plt.title('2016-2019 (15-day step) median NDRE and percentiles for Manaus Tile')
median = plt.plot(x,y, color = "black", linestyle = ":", label = "Median")
plt.legend(loc = "lower right")
plt.xticks(ticks = [0, 25, 50, 75, 99], labels = ['Jan 2016', 'Jan 2017', 'Jan 2018', 'Jan 2019', '26 Dec 2019'])
plt.show()

In [ ]:
#Atto
#Mean and SD
x = period
y = mean_ndre
m = median_ndre
plt.axes([2, 0, 2, 1])
plt.ylim(bottom = 0, top = 1)
plt.grid(b = True, which = 'major', axis = 'x')
plt.title('2016-2019 (15-day step) mean, median, and SD of NDRE for Atto Tile')
plt.fill_between(x, plus_error, minus_error, color = "grey", alpha = .5, label = "Standard Deviation")
mean = plt.plot(x,y, color = "black", label = "Mean")
median = plt.plot(x,m, color = "red", label = "Median")
plt.legend(loc = "lower right")
plt.xticks(ticks = [0, 25, 50, 75, 99], labels = ['Jan 2016', 'Jan 2017', 'Jan 2018', 'Jan 2019', '26 Dec 2019'])
plt.show()

In [ ]:
#Atto
#Median and Percentiles
x = period
plt.axes([2, 0, 2, 1])
plt.ylim(bottom = 0, top = 1)
plt.grid(b = True, which = 'major', axis = 'x')
y5 = plt.plot(x,percentile_ndre[0], color = "red", label = "P5")
y10 = plt.plot(x,percentile_ndre[1], color = "orange", label = "P10")
y25 = plt.plot(x,percentile_ndre[2], color = "green", label = "P25")
y75 = plt.plot(x,percentile_ndre[3], color = "blue", label = "P75")
y90 = plt.plot(x,percentile_ndre[4], color = "purple", label = "P90")
y95 = plt.plot(x,percentile_ndre[5], color = "black", label = "P95")
plt.title('2016-2019 (15-day step) median NDRE and percentiles for Atto Tile')
median = plt.plot(x,m, color = "black", linestyle = ":", label = "Median")
plt.legend(loc = "lower right")
plt.xticks(ticks = [0, 25, 50, 75, 99], labels = ['Jan 2016', 'Jan 2017', 'Jan 2018', 'Jan 2019', '26 Dec 2019'])
plt.show()

In [ ]:
#Corumba
#Mean and SD
x = period
y = mean_ndre
m = median_ndre
plt.axes([2, 0, 2, 1])
plt.ylim(bottom = 0, top = 1)
plt.grid(b = True, which = 'major', axis = 'x')
plt.title('2016-2019 (15-day step) mean, median, and SD of NDRE for Corumba Tile')
plt.fill_between(x, plus_error, minus_error, color = "grey", alpha = .5, label = "Standard Deviation")
mean = plt.plot(x,y, color = "black", label = "Mean")
median = plt.plot(x,m, color = "red", label = "Median")
plt.legend(loc = "lower right")
plt.xticks(ticks = [0, 25, 50, 75, 99], labels = ['Jan 2016', 'Jan 2017', 'Jan 2018', 'Jan 2019', '26 Dec 2019'])
plt.show()

In [ ]:
#Corumba
#Median and Percentiles
x = period
plt.axes([2, 0, 2, 1])
plt.ylim(bottom = 0, top = 1)
plt.grid(b = True, which = 'major', axis = 'x')
y5 = plt.plot(x,percentile_ndre[0], color = "red", label = "P5")
y10 = plt.plot(x,percentile_ndre[1], color = "orange", label = "P10")
y25 = plt.plot(x,percentile_ndre[2], color = "green", label = "P25")
y75 = plt.plot(x,percentile_ndre[3], color = "blue", label = "P75")
y90 = plt.plot(x,percentile_ndre[4], color = "purple", label = "P90")
y95 = plt.plot(x,percentile_ndre[5], color = "black", label = "P95")
plt.title('2016-2019 (15-day step) median NDRE and percentiles for Corumba Tile')
median = plt.plot(x,m, color = "black", linestyle = ":", label = "Median")
plt.legend(loc = "lower right")
plt.xticks(ticks = [0, 25, 50, 75, 99], labels = ['Jan 2016', 'Jan 2017', 'Jan 2018', 'Jan 2019', '26 Dec 2019'])
plt.show()

In [ ]:
#Mato
#Mean and SD
x = period
y = mean_ndre
m = median_ndre
plt.axes([2, 0, 2, 1])
plt.ylim(bottom = 0, top = 1)
plt.grid(b = True, which = 'major', axis = 'x')
plt.title('2016-2019 (15-day step) mean, median, and SD of NDRE for Mato Tile')
plt.fill_between(x, plus_error, minus_error, color = "grey", alpha = .5, label = "Standard Deviation")
mean = plt.plot(x,y, color = "black", label = "Mean")
median = plt.plot(x,m, color = "red", label = "Median")
plt.legend(loc = "lower right")
plt.xticks(ticks = [0, 25, 50, 75, 99], labels = ['Jan 2016', 'Jan 2017', 'Jan 2018', 'Jan 2019', '26 Dec 2019'])
plt.show()

In [ ]:
#Mato
#Median and Percentiles
x = period
plt.axes([2, 0, 2, 1])
plt.ylim(bottom = 0, top = 1)
plt.grid(b = True, which = 'major', axis = 'x')
y5 = plt.plot(x,percentile_ndre[0], color = "red", label = "P5")
y10 = plt.plot(x,percentile_ndre[1], color = "orange", label = "P10")
y25 = plt.plot(x,percentile_ndre[2], color = "green", label = "P25")
y75 = plt.plot(x,percentile_ndre[3], color = "blue", label = "P75")
y90 = plt.plot(x,percentile_ndre[4], color = "purple", label = "P90")
y95 = plt.plot(x,percentile_ndre[5], color = "black", label = "P95")
plt.title('2016-2019 (15-day step) median NDRE and percentiles for Mato Tile')
median = plt.plot(x,m, color = "black", linestyle = ":", label = "Median")
plt.legend(loc = "lower right")
plt.xticks(ticks = [0, 25, 50, 75, 99], labels = ['Jan 2016', 'Jan 2017', 'Jan 2018', 'Jan 2019', '26 Dec 2019'])
plt.show()

In [ ]:
#Saop
#Mean and SD
x = period
y = mean_ndre
m = median_ndre
plt.axes([2, 0, 2, 1])
plt.ylim(bottom = 0, top = 1)
plt.grid(b = True, which = 'major', axis = 'x')
plt.title('2016-2019 (15-day step) mean, median, and SD of NDRE for Saop Tile')
plt.fill_between(x, plus_error, minus_error, color = "grey", alpha = .5, label = "Standard Deviation")
mean = plt.plot(x,y, color = "black", label = "Mean")
median = plt.plot(x,m, color = "red", label = "Median")
plt.legend(loc = "lower right")
plt.xticks(ticks = [0, 25, 50, 75, 99], labels = ['Jan 2016', 'Jan 2017', 'Jan 2018', 'Jan 2019', '26 Dec 2019'])
plt.show()

In [ ]:
#Saop
#Median and Percentiles
x = period
plt.axes([2, 0, 2, 1])
plt.ylim(bottom = 0, top = 1)
plt.grid(b = True, which = 'major', axis = 'x')
y5 = plt.plot(x,percentile_ndre[0], color = "red", label = "P5")
y10 = plt.plot(x,percentile_ndre[1], color = "orange", label = "P10")
y25 = plt.plot(x,percentile_ndre[2], color = "green", label = "P25")
y75 = plt.plot(x,percentile_ndre[3], color = "blue", label = "P75")
y90 = plt.plot(x,percentile_ndre[4], color = "purple", label = "P90")
y95 = plt.plot(x,percentile_ndre[5], color = "black", label = "P95")
plt.title('2016-2019 (15-day step) median NDRE and percentiles for Saop Tile')
median = plt.plot(x,m, color = "black", linestyle = ":", label = "Median")
plt.legend(loc = "lower right")
plt.xticks(ticks = [0, 25, 50, 75, 99], labels = ['Jan 2016', 'Jan 2017', 'Jan 2018', 'Jan 2019', '26 Dec 2019'])
plt.show()

In [ ]:
#Export images from a collection
out_dir = '/home/6ru/tiles/manaus/'
geemap.ee_export_image_collection(col, out_dir, region = manaus, scale = 20, file_per_band=False)